In [127]:
import pandas as pd
import os
import glob
from itertools import chain
import openpyxl
import csv
from collections import namedtuple
from chardet.universaldetector import UniversalDetector
import requests
import hashlib
import io
import warnings
from typing import Literal

In [128]:
Fileinfo = namedtuple('Fileinfo', ['delimiter', 'encoding'])

class Sha256Mismatch(Exception):
    pass

class DataFile:
    def __init__(
            self,
            file,
            filename = None,
            sha_integrity:Literal['strict', 'check', 'ignore'] = 'strict',
            column_integrity:Literal['strict', 'check', 'ignore'] = 'strict'
            ):
        
        if sha_integrity not in ['strict', 'check', 'ignore']:
            raise AttributeError("The `sha_integrity` parameter must be specified as either 'strict', 'check', or 'ignore'.")
        if column_integrity not in ['strict', 'check', 'ignore']:
            raise AttributeError("The `sha_integrity` parameter must be specified as either 'strict', 'check', or 'ignore'.")
        if type(file) not in [pd.DataFrame, str]:
            raise AttributeError("The `file` parameter must be a Pandas DataFrame or a valid filepath that can be read into a DataFrame.")
        if type(file) == pd.DataFrame & type(filename) == None:
            raise AttributeError("When `file` is a Pandas DataFrame you must also supply a filename.")
        
        if type(file) == pd.DataFrame:
            self.DataFrame = file
            self.Filename = filename
        elif type(file) == str:
            self.Filename = file
            self.read(file)

        if sha_integrity == 'strict':
            self.ensure_sha256_match()
        elif sha_integrity == 'check':
            self.ensure_sha256_match(strict=False)

        if column_integrity == 'strict':
            self.ensure_column_integrity()
        elif column_integrity == 'check':
            self.ensure_column_integrity(strict=False)

    def read(self, filename:str) -> None:
        self.DataFrame = pd.read_excel(filename)

    def get_file_encoding(self, file_path: str) -> str:
        detector = UniversalDetector()
        for line in open(file_path, 'rb'):
            detector.feed(line)
            if detector.done: break
        detector.close()
        result = detector.result['encoding']
        return result

    def get_file_delimiter(self, file_path: str, encoding: str) -> str:
        with open(file_path, 'r', encoding=encoding) as file:
            delimiter = str(csv.Sniffer().sniff(file.read()).delimiter)

    def get_file_info(self, file_path: str) -> namedtuple:
        encoding = self.get_file_encoding(file_path)
        delimiter = self.get_file_delimiter(file_path, encoding)
        return Fileinfo(delimiter, encoding)
    
    def ensure_sha256_match(self, strict:bool = True) -> bool:
        file = self.filename.split('/')[-1]

        # GET the file from NIH RePORT
        url = "https://report.nih.gov/award/files/" + file
        session = requests.session()
        response = session.get(url, stream=True)
        nih_file = io.BytesIO(response.content)
        nih_digest = hashlib.file_digest(nih_file, "sha256")

        # Create hash for local file (in repository)
        with open(self.filename, 'rb') as local_file:
            local_digest = hashlib.file_digest(local_file, "sha256")

        if strict and nih_digest.hexdigest() != local_digest.hexdigest() :
            raise Sha256Mismatch(f'The local file {file} does not match the corresponding file downloaded from NIH.')
        else:
            # This will return True if the repository file exactly matches the file downloaded from NIH
            return nih_digest.hexdigest() == local_digest.hexdigest()
        
    def ensure_column_integrity(self, strict:bool = True):
        expected_columns = pd.Index(['ORGANIZATION NAME', 'ORGANIZATION ID (IPF)', 'PROJECT NUMBER',
                                     'FUNDING MECHANISM', 'NIH REFERENCE', 'PI NAME', 'PI PERSON ID',
                                     'PROJECT TITLE', 'DEPT NAME', 'NIH DEPT COMBINING NAME',
                                     'NIH MC COMBINING NAME', 'DIRECT COST', 'INDIRECT COST', 'FUNDING',
                                     'CONGRESSIONAL DISTRICT', 'CITY', 'STATE OR COUNTRY NAME', 'ZIP CODE',
                                     'ATTRIBUTED TO MEDICAL SCHOOL', 'MEDICAL SCHOOL LOCATION',
                                     'INSTITUTION TYPE', 'AWARD NOTICE DATE', 'OPPORTUNITY NUMBER'])
        diff_columns = self.df.keys().difference(expected_columns)
        column_errors = diff_columns.to_list()
        error_string = ', '.join(column_errors)
        if (len(column_errors) > 0) and strict:
            error_values = self.df[[diff_columns.to_list()[0]]].dropna()
            self.df.drop(index=error_values.index, inplace=True)
            self.df.drop(columns=error_values, inplace=True)
        elif (len(column_errors) > 0) and not(strict):
            warnings.warn(f"DataFile has extraneous columns: {error_string}\n\nThese are being ignored", UserWarning)
        
        return df

In [105]:
exts = [".csv", ".xls", ".xlsx", ".tsv"]
files = list(chain.from_iterable([glob.glob('./data/*' + ext) for ext in exts]))

In [110]:
data = []
failed = []

for i in files:
    try:
        ensure_sha256_match(i)
        df = pd.read_excel(i)
        df['FILENAME'] = i
        df = ensure_column_integrity(df)
        data.append(df)
    except Sha256Mismatch as e:
        print(f'{e}\n\nExecution will stop.  Download the NIH file and replace locally before processing.')
    except Exception as e:
        print(f'Failed to parse the file located at {i}.  Will attempt automatic decoding...')
        try:
            info = get_file_info(i)
            df = pd.read_csv(i, sep=info.delimiter, encoding=info.encoding, engine="python")
            df['FILENAME'] = i
            df = ensure_column_integrity(df)
            data.append(df)
            print('Automatic decoding succeeded.')
        except Exception as e:
            print(f'Failed to automatically detect and parse the file located at {i}.\n\n{e}')
            failed.append(i)
            continue
        continue

all_fy = pd.concat(data)

/var/folders/0g/f36t65452jq_fxt8txzb439c0000gn/T/ipykernel_87784/1300396238.py:38: UserWarning: File has extraneous columns: Unnamed: 23
  warnings.warn(f"File has extraneous columns: {error_string}", UserWarning)
/var/folders/0g/f36t65452jq_fxt8txzb439c0000gn/T/ipykernel_87784/1300396238.py:38: UserWarning: File has extraneous columns: Unnamed: 23
  warnings.warn(f"File has extraneous columns: {error_string}", UserWarning)


Failed to parse the file located at ./data/Worldwide2008.xls.  Will attempt automatic decoding...
Automatic decoding succeeded.


/var/folders/0g/f36t65452jq_fxt8txzb439c0000gn/T/ipykernel_87784/1300396238.py:38: UserWarning: File has extraneous columns: Unnamed: 23
  warnings.warn(f"File has extraneous columns: {error_string}", UserWarning)
/var/folders/0g/f36t65452jq_fxt8txzb439c0000gn/T/ipykernel_87784/1300396238.py:38: UserWarning: File has extraneous columns: Unnamed: 23
  warnings.warn(f"File has extraneous columns: {error_string}", UserWarning)


Failed to parse the file located at ./data/Worldwide2010.xls.  Will attempt automatic decoding...
Automatic decoding succeeded.


/var/folders/0g/f36t65452jq_fxt8txzb439c0000gn/T/ipykernel_87784/1300396238.py:38: UserWarning: File has extraneous columns: Unnamed: 23
  warnings.warn(f"File has extraneous columns: {error_string}", UserWarning)
